<a href="https://colab.research.google.com/github/aritrasen87/LLM/blob/main/Qwen2_5_GRPO_Training_Reasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installations**

In [1]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [2]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-07 11:42:25 __init__.py:190] Automatically detected platform cuda.


In [3]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

==((====))==  Unsloth 2025.2.4: Fast Qwen2 patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 49.66%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 4.9 GB. Also swap space = 2 GB.
WARNING 02-07 11:42:42 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-07 11:43:06 config.py:542] This model supports multiple tasks: {'reward', 'classify', 'generate', 'embed', 'score'}. Def

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-07 11:43:26 model_runner.py:1115] Loading model weights took 2.2160 GB
INFO 02-07 11:43:26 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 02-07 11:43:29 worker.py:267] Memory profiling takes 3.27 seconds
INFO 02-07 11:43:29 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.50) = 7.32GiB
INFO 02-07 11:43:29 worker.py:267] model weights take 2.22GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.05GiB; the rest of the memory reserved for KV Cache is 4.01GiB.
INFO 02-07 11:43:30 executor_base.py:110] # CUDA blocks: 7300, # CPU blocks: 3640
INFO 02-07 11:43:30 executor_base.py:115] Maximum concurrency for 1024 tokens per request: 114.06x
INFO 02-07 11:43:32 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error oc

Capturing CUDA graph shapes: 100%|██████████| 27/27 [00:54<00:00,  2.02s/it]

INFO 02-07 11:44:26 model_runner.py:1562] Graph capturing finished in 55 secs, took 0.62 GiB
INFO 02-07 11:44:26 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 60.63 seconds



Unsloth 2025.2.4 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### System Prompt Design

In [4]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<think>
...
</think>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<think>
{reasoning}
</think>
<answer>
{answer}
</answer>
"""

### Data prep

In [5]:
data = load_dataset('openai/gsm8k', 'main')['train']

In [6]:
count = 0
for i in data:
    if count==5:
        break
    else:
        print(f"Question: {i['question']}")
        print(f"Answer: {i['answer']}")
        print("\n")
    count+=1

Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Answer: Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72


Question: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
Answer: Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.
Working 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.
#### 10


Question: Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
Answer: In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50.
Betty's grandparents gave her 15 * 2 = $<<15*2=30>>30.
This means, Betty needs 100 - 50 - 30 - 15 = $<<100-

In [7]:
def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

In [8]:
def get_gsm8k_questions(data) -> Dataset:
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions(data)

In [9]:
count = 0
for i in dataset:
    if count==2:
        break
    else:
        print(f"Question: {i['prompt']}")
        print(f"Answer: {i['answer']}")
        print("\n")
    count+=1

Question: [{'content': '\nRespond in the following format:\n<think>\n...\n</think>\n<answer>\n...\n</answer>\n', 'role': 'system'}, {'content': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'role': 'user'}]
Answer: 72


Question: [{'content': '\nRespond in the following format:\n<think>\n...\n</think>\n<answer>\n...\n</answer>\n', 'role': 'system'}, {'content': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?', 'role': 'user'}]
Answer: 10




In [10]:
def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

### Reward Functions

In [11]:
# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<think>\n.*?\n</think>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<think>.*?</think>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<think>\n") == 1:
        count += 0.125
    if text.count("\n</think>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

### Training arguments setup

Now set up GRPO Trainer and all configurations!

In [12]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

In [13]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 250
 "-____-"     Number of trainable parameters = 119,734,272


-------------------- Question:
Ahmed and Emily are having a contest to see who can get the best grade in the class. There have been 9 assignments and Ahmed has a 91 in the class. Emily has a 92. The final assignment is worth the same amount as all the other assignments. Emily got a 90 on the final assignment. What is the minimum grade Ahmed needs to get to beat Emily if all grades are whole numbers? 
Answer:
100 
Response:
<think>
Let's break down the problem step-by-step.

First, we need to calculate the total points both Ahmed and Emily have earned so far, excluding the final assignment.

Ahmed has scored an average of 91 over 9 assignments. Thus, the total points Ahmed has so far is:
\[ \text{Total Points Ahmed} = 91 \times 9 = 819 \]

Emily has scored an average of 92 over 9 assignments. Thus, the total points Emily has so far is:
\[ \text{Total Points Emily} = 92 \times 9 = 828 \]

Emily got a 90 on her final assignment. Therefore, her total points after the final assignment is:
\

Step,Training Loss,reward,reward_std,completion_length,kl
1,0.000000,0.042625,0.232992,200.000000,0.000000
2,-0.000000,-0.203375,0.205280,200.000000,0.000000
3,0.000000,-0.163875,0.165877,191.125000,0.000009
4,0.000000,0.060375,0.182787,200.000000,0.000008
5,0.000000,0.125000,0.000000,200.000000,0.000007
6,0.000000,-0.163625,0.197656,189.875000,0.000017
7,0.000000,-0.175750,0.180227,167.375000,0.000011
8,0.000000,-0.278625,0.119768,195.500000,0.000007
9,0.000000,-0.064375,0.224298,194.875000,0.000013
10,0.000000,-0.049625,0.152416,143.875000,0.000014


Streaming output truncated to the last 5000 lines.
</think>
<answer>
First, calculate the total number of seats available:

\[
15 \text{ tables} \times 10 \text{ seats per table} = 150 \text{ seats}
\]

Since 1/10 of the seats are left unseated, 9/10 of the seats are taken. To find out how many seats are 
Extracted:
First, calculate the total number of seats available:

\[
15 \text{ tables} \times 10 \text{ seats per table} = 150 \text{ seats}
\]

Since 1/10 of the seats are left unseated, 9/10 of the seats are taken. To find out how many seats are
-------------------- Question:
Fiona completed 36 math questions in an hour. Shirley was able to complete twice as many math questions within that same time, and Kiana completed half of the sum of Fiona and Shirley's math questions. If they each did the same number of questions the following hour, how many math questions did all three girls complete in 2 hours? 
Answer:
324 
Response:
<think>
First, we calculate the number of questions Shirl

TrainOutput(global_step=250, training_loss=0.004461719964464919, metrics={'train_runtime': 5218.098, 'train_samples_per_second': 0.048, 'train_steps_per_second': 0.048, 'total_flos': 0.0, 'train_loss': 0.004461719964464919})

### Inference with base model

In [22]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None, ## no grpo lora
)[0].outputs[0].text

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.41it/s, est. speed input: 52.62 toks/s, output: 22.75 toks/s]


In [23]:
print(output)

There are no letters "r" in the word "strawberry".


### Inference with GRPO based model

Saving the LoRA GRPO

In [14]:
model.save_lora("Qwen_grpo_lora")

In [25]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("Qwen_grpo_lora"),
)[0].outputs[0].text

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.62s/it, est. speed input: 25.38 toks/s, output: 26.62 toks/s]


In [26]:
print(output)

<think>
In the word "strawberry," there is only one instance of the letter 'r.' It appears in the middle of the word.
</think>
<answer>
1
</answer>

